## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
credit_data = pd.read_csv('/datasets/data.csv')
print(credit_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


В базе данных хранится 12 колонок различных типов данных с информацией о 21525 клиентах, которые брали кредиты в течение какого-то времени. Видно, что в столбцах **"days_employed"** и **"total_income"** имеются пропущенные значения. Также следует отметить, что вышеперечисденные столбцы содержат **количественные** переменные

Выведем первые 10 строк таблицы:

In [2]:
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце "days_employed" имеются отрицательные данные, в "education" используется различный регистр, а в "purpose" одинаковые цели описаны по-разному.

Найдем уникальные значения в столбцах "gender" и "dob_years":

In [3]:
credit_data['gender'].unique()


array(['F', 'M', 'XNA'], dtype=object)

In [4]:
credit_data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В столбце "gender" есть неуказанные данные, но типа object ("XNA").

Отсортируем таблицу по невозрастанию по значениям столбца "days_employed" и узнаем, содержит ли таблица "вбросы"

In [5]:
credit_data.sort_values('days_employed',ascending=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Видим, что в таблице есть явные вбросы (стаж некоторых людей превышает 1000 лет)

### Вывод

Каждая строка таблицы содержит информацию о клиенте, который взял кредит. Судя по задачам заказчика, самыми важными стобцами являются **"children"** - количество детей в семье, **"family_status"** - семейное положение и **"debt"** - имелась ли задолженность по возврату кредита.

В спектр проблем входят:
* отрицательные значения столбца "days_employed"
* вохможные дубликаты
* различный регистр в столбце "education"
* значение "XNA" в столбце "gender"
* пропуски значений в столбцах "days_employed" и "total_income"
* вбросы в столбце "days_employed"
* вещественный тип данных в в столбце "days_employed"

Кроме того, заметим, что люди, которые не заполняли стаж, так же не указывали свой заработок.

### Шаг 2. Предобработка данных

### Обработка пропусков

Узнаем количество "XNA" в столбце "gender" и удалим эти значения

In [6]:
credit_data[credit_data['gender'] == 'XNA']['gender'].count()

1

In [7]:
credit_data = credit_data[credit_data['gender'] != 'XNA']

Заполним неуказанные данные по следующему алгоритму:

* уберем дубликаты в столбце "education" методом .str.lower()
* методом groupby сгруппируем таблицу по столбцам 'gender', 'education', 'income_type', 'family_status'
* узнаем медианы зарплат для людей с разными показателями. данных столбоцов
* заменим неуказанные значения на соответствующие медианы

Для реализации последних двух пунктов потребуется метод transform, который заменяет значение на то, которое получилось в результате выполнения указанной функции

In [8]:
credit_data['education'] = credit_data['education'].str.lower()
credit_data['total_income'] = credit_data.groupby(['gender', 'education', 'income_type', 'family_status'])['total_income'].transform(lambda x: x.fillna(x.mean()))

In [9]:
credit_data.sort_values('total_income', ascending=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,2.120528e+04,заняться высшим образованием
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,2.066726e+04,недвижимость
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
12407,0,NaN,57,неоконченное высшее,2,в разводе,3,F,пенсионер,0,NaN,получение высшего образования


Значения столбца "days_employed" не являются основополагающими для ответа на поставленные вопросы, поэтому заменим неуказанные значения на нули методом fillna()

In [10]:
credit_data['days_employed'] = credit_data['days_employed'].fillna(0)

In [11]:
credit_data['days_employed'].isna().sum()

0

In [12]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21521 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Осталось несколько незаполненных значения в столбце "total_income"
Удалим их методом dropna()

In [13]:
credit_data = credit_data.dropna()
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21521 entries, 0 to 21524
Data columns (total 12 columns):
children            21521 non-null int64
days_employed       21521 non-null float64
dob_years           21521 non-null int64
education           21521 non-null object
education_id        21521 non-null int64
family_status       21521 non-null object
family_status_id    21521 non-null int64
gender              21521 non-null object
income_type         21521 non-null object
debt                21521 non-null int64
total_income        21521 non-null float64
purpose             21521 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

 В результате данные столбца "total_income" были заменены на соответствующие медианные значения, а неуказанные значения, относящиеся к столбцу "days_employed", были заменены на нули.

В итоге, через метод info() можем убедиться, что в наших данных нет неуказанных данных.

### Замена типа данных

В нашей таблице данные в столбце "days_employed" представлены в вещественном виде, изменим его на целый:

In [ ]:
credit_data['days_employed'] = credit_data['days_employed'].astype('int')
credit_data.head()

При переводе из вещественного типа переменных в целочисленный в pandas используется метод .astype()

### Вывод

Данные столбца "days_employed" стали относиться к целочисленному типу переменных.

### Обработка дубликатов

Узнаем, были ли в таблице строки с абсолютно. схожими данными:

In [15]:
credit_data = credit_data.drop_duplicates()

In [16]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21450 entries, 0 to 21524
Data columns (total 12 columns):
children            21450 non-null int64
days_employed       21450 non-null int64
dob_years           21450 non-null int64
education           21450 non-null object
education_id        21450 non-null int64
family_status       21450 non-null object
family_status_id    21450 non-null int64
gender              21450 non-null object
income_type         21450 non-null object
debt                21450 non-null int64
total_income        21450 non-null float64
purpose             21450 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Вывод

Как видим, в. таблице были полностью схожие строки, что, вероятнее всего, говорит о том, что некоторые данные были записаны повторно.
Методом drop_duplicates мы удалили одинаковые строки.

Возможно, у некоторых людей проблемы с интернетом, и они не поняли отправилась ли форма, поэтому они заполнили ее несколько раз.

### Лемматизация

Для лемматизации используем библиотеку **pymystem3**.

Цели получения кредита различаются не сильно:
* жилье
* автомобиль
* образование
* свадьба
* недвижимость
Заменим фразы в столбце "purpose" на вышеперечисленные слова,  для этого используем функцию **change_purpose** которая лемматизирует фразу и перебирает возможные варианты:

In [17]:
from pymystem3 import Mystem
m = Mystem()
def change_purpose(purpose):
        phrase = m.lemmatize(purpose)
        if 'жилье' in phrase:
            return 'жилье'
        elif 'автомобиль' in phrase:
            return 'автомобиль'
        elif 'образование'in phrase:
            return 'образование'
        elif 'свадьба'in phrase:
            return 'свадьба'
        elif 'недвижимость'in phrase:
            return 'недвижимость'
        
credit_data['purpose'] = credit_data['purpose'].apply(change_purpose)
credit_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,жилье
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,автомобиль
21522,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,автомобиль


Видим, что цели на взятие кредита, в целом, не сильно различаются.

### Вывод

Усреднили значения столбца "purpose", используя библиотеку pymystem3.
Для этого создали функцию **change_purpose**, которая меняет произвольное название, данное клиентом, на заранее подготовленное нами.

### Категоризация данных

Заменим количество детей на категории: 
* бездетная
* малодетная
* многодетная

Используя функцию family_mem() и метод .apply() категоризируем данные столбца "children" на тип семьи по количеству детей.
Далее меняем название столбца "children" на "family_category"

In [18]:
def family_mem(num_children):
    if num_children < 1:
        return "бездетная"
    elif num_children < 3:
        return "малодетная"
    else:
        return "многодетная"
credit_data['children'] = credit_data['children'].apply(family_mem)
credit_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,малодетная,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,жилье
1,малодетная,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль
2,бездетная,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,жилье
3,многодетная,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование
4,бездетная,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,малодетная,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,жилье
21521,бездетная,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,автомобиль
21522,малодетная,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,многодетная,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,автомобиль


Переименуем столбец children, т.к он сменил свое значение:

In [19]:
credit_data.set_axis(['family_category', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status',
                     'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'],
                     inplace=True, axis='columns')
credit_data.head()

,family_category,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,малодетная,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,жилье
1,малодетная,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль
2,бездетная,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,жилье
3,многодетная,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование
4,бездетная,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба


In [20]:
def salary(x):
    if x < 80000:
        return 'x<80'
    elif x < 100000:
        return '80<x<100'
    elif x < 200000:
        return '100<x<200'
    elif x < 300000:
        return '200<x<300'
    elif x < 400000:
        return '300<x<400'
    else:
        return 'x>400'
    
credit_data['level_salary'] = credit_data['total_income'].apply(salary)

In [21]:
credit_data.head()

,family_category,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,level_salary
0,малодетная,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,жилье,200<x<300
1,малодетная,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль,100<x<200
2,бездетная,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,жилье,100<x<200
3,многодетная,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование,200<x<300
4,бездетная,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба,100<x<200


### Вывод

По итогу, мы добавили новый столбец "level_salary" с категориями зарплат, изменили название столбца "children" на "family_category" и категоризовали его

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
credit_data.groupby('family_category')['debt'].mean().sort_values(ascending=False)

family_category
малодетная     0.093003
многодетная    0.085526
бездетная      0.075279
Name: debt, dtype: float64

### Вывод

По итогу получили, что там, где нет детей и много детей - показатель лучше, чем в семьях в которых мало детей. Следовательно, можно сделать вывод, что количество детей - не показатель платежеспособности клиента.

Следовательно, **зависимости нет**.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
credit_data.groupby('family_status')['debt'].mean().sort_values(ascending=False)

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093494
женат / замужем          0.075458
в разводе                0.071189
вдовец / вдова           0.065762
Name: debt, dtype: float64

### Вывод

Как можем заметить, те, кто относится к категории "женат / не замужем" чаще не справляются с кредитом, чем семейные люди. Более того, судя по статистике, такой тип людей является самым проблемным в сфере кредитования.
В свою очередь, люди, находящиеся в браке, показывают значительно более дисциплинировананные показатели.

Следовательно, **зависимости есть**.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
credit_data.groupby('level_salary')['debt'].mean().sort_values(ascending=False)

level_salary
100<x<200    0.087183
80<x<100     0.082305
300<x<400    0.078288
x<80         0.076416
200<x<300    0.069528
x>400        0.058712
Name: debt, dtype: float64

### Вывод

Если предположить, что чем больше человек получает, тем более вероятно, что он будет исправно платить кредит, то мы окажемся неправы.

С одной стороны, люди, получающие более 400 тыс. в месяц, платят лучше всех, но с другой, самую последюю позицию таблицы заняли кредиторы не с самым маленьким доходом (которые в свою очередь входят в тройку!)

Следовательно, можно сделать вывод, что **нет зависимости** между уровнем дохода и возвратом кредита в срок

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
credit_data.groupby('purpose')['debt'].mean().sort_values(ascending=False)

purpose
автомобиль      0.093590
образование     0.092223
свадьба         0.080034
недвижимость    0.074657
жилье           0.069074
Name: debt, dtype: float64

### Вывод

Логично, что лучше всего люди платят кредиты на жилье, крыша над головой - первая необходимость, автомобиль же не является необходимым предметом для современного человека, поэтому и тендеция просрачивать автокредиты достаточно популярна.
В нынешних реалиях, то что человек имеет высшее образование не гарантирует ему, что он будет кредитоспособен, кроме того, в силу своего возраста, люди, относящиеся к данной категории являются не самыми дисциплинированными.
После свадьбы многие разводятся в течение года. Договариваясь платить кредит вместе, вскоре они расходятся и только один человек пытается выплатить кредит. Это не всегда получается, о чем говорит статистика - кредиты на свадьбы входят в "тройку риска"


### Шаг 4. Общий вывод

По итогу выяснили, что самое очевидное предположение о том, что чем больше зарплата, тем более платежеспособен клиент, не является правдой.
Вероятнее всего, кредит выплатит человек, берущий его на жилье, с зарплатой более 400 тыс. рублей, который является вдовцом и у него нет детей.
Мы предобработали данные и, опираясь на них, сделали вышеперечисленные выводы.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.